In [ ]:
import pygame
import random
import math

# Constants
WIDTH = 800
HEIGHT = 600
BOID_COUNT = 50
BOID_SPEED = 4
MAX_FORCE = 0.1
NEIGHBOR_RADIUS = 50
SEPARATION_RADIUS = 25
COUPLING_STRENGTH = 0.5


def distance(pos1, pos2):
    return math.sqrt((pos1.x - pos2.x) ** 2 + (pos1.y - pos2.y) ** 2)


class Boid:
    def __init__(self, x, y):
        self.position = pygame.math.Vector2(x, y)
        self.velocity = pygame.math.Vector2(random.uniform(-1, 1), random.uniform(-1, 1))
        self.velocity.scale_to_length(BOID_SPEED)
        self.acceleration = pygame.math.Vector2(0, 0)
        self.phase = random.uniform(-math.pi, math.pi)  # Phase for Kuramoto model
        self.natural_frequency = random.uniform(-0.1, 0.1)  # Natural frequency

    def update(self):
        self.velocity += self.acceleration
        self.position += self.velocity
        self.acceleration *= 0  # Reset acceleration

    def apply_force(self, force):
        self.acceleration += force

    def edges(self):
        if self.position.x > WIDTH:
            self.position.x = 0
        elif self.position.x < 0:
            self.position.x = WIDTH
        if self.position.y > HEIGHT:
            self.position.y = 0
        elif self.position.y < 0:
            self.position.y = HEIGHT

    def flock(self, boids):
        apply_rules(self, boids)
        apply_kuramoto(self, boids, COUPLING_STRENGTH)

    def draw(self, screen):
        angle = self.velocity.angle_to(pygame.math.Vector2(1, 0))
        points = [
            self.position + pygame.math.Vector2(10, 0).rotate(angle),
            self.position + pygame.math.Vector2(-10, 5).rotate(angle),
            self.position + pygame.math.Vector2(-10, -5).rotate(angle),
        ]
        pygame.draw.polygon(screen, (255, 255, 255), points)


def apply_kuramoto(boid, boids, coupling_strength):
    phase_coupling = 0
    nearby_boids = 0

    for other in boids:
        if other == boid:
            continue
        if distance(boid.position, other.position) < NEIGHBOR_RADIUS:
            nearby_boids += 1
            phase_coupling += math.sin(other.phase - boid.phase)

    if nearby_boids > 0:
        # Update phase using natural frequency and coupling
        boid.phase += boid.natural_frequency + (coupling_strength / nearby_boids) * phase_coupling
        # Keep phase within [-π, π] for simplicity
        boid.phase = (boid.phase + math.pi) % (2 * math.pi) - math.pi

    # Update velocity to align with the new phase
    boid.velocity = pygame.math.Vector2(math.cos(boid.phase), math.sin(boid.phase)) * BOID_SPEED


def apply_rules(boid, boids):
    separation_force = pygame.math.Vector2(0, 0)
    alignment_force = pygame.math.Vector2(0, 0)
    cohesion_force = pygame.math.Vector2(0, 0)
    nearby_boids = 0

    for other in boids:
        if other == boid:
            continue
        dist = distance(boid.position, other.position)
        if dist < NEIGHBOR_RADIUS:
            nearby_boids += 1
            alignment_force += other.velocity
            cohesion_force += other.position
            if dist < SEPARATION_RADIUS:
                separation_force += boid.position - other.position

    if nearby_boids > 0:
        # Normalize and average forces
        alignment_force = (alignment_force / nearby_boids).normalize() * BOID_SPEED
        cohesion_force = ((cohesion_force / nearby_boids) - boid.position).normalize() * BOID_SPEED
        if separation_force.length() > 0:  # Avoid zero-length vector normalization
            separation_force = separation_force.normalize() * BOID_SPEED

    # Combine forces with weights
    boid.velocity += separation_force * 1.5 + alignment_force * 1.0 + cohesion_force * 1.0
    boid.velocity = boid.velocity.normalize() * BOID_SPEED  # Ensure consistent speed


# Pygame setup
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

# Create boids
boids = [Boid(random.randint(0, WIDTH), random.randint(0, HEIGHT)) for _ in range(BOID_COUNT)]

# Main loop
running = True
while running:
    screen.fill((0, 0, 0))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    for boid in boids:
        boid.edges()
        boid.flock(boids)
        boid.update()
        boid.draw(screen)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()


KeyboardInterrupt: 

: 